In [1]:
import json
from datetime import datetime
import time
from threading import Thread,Lock

In [2]:
#Inizializzo id number
id_num=1
#Definisco semaforo
mutex=Lock()

In [3]:
#Assegna sezione censo
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

with open('/Volumes/HDD/Università/Tirocinio/Dati censo/sezioni_censimento_2011.geojson') as file:
    gj_sezioni = json.load(file)
features_sezioni=gj_sezioni['features']

In [4]:
def assegna_censo(coord_X,coord_Y):
    point=Point(coord_X,coord_Y)
    for f in features_sezioni:
        polygon = Polygon(f['geometry']['coordinates'][0])
        if polygon.contains(point):
            return f['properties']['SEZ']

In [5]:
#Importo i Geojson delle 4 macro categorie
with open('../Geojson/shop.geojson') as file:
    gj_shop = json.load(file)
    shop = gj_shop['features']
with open('../Geojson/amenity.geojson') as file:
    gj_amenity = json.load(file)
    amenity = gj_amenity['features']
with open('../Geojson/leisure.geojson') as file:
    gj_leisure = json.load(file)
    leisure = gj_leisure['features']
with open('../Geojson/building.geojson') as file:
    gj_building = json.load(file)
    building = gj_building['features']

In [7]:
#Importo categorie
#json SHOP
with open('../Categorie/shop_essenziale.json') as file:
    cat_shop_essenziale = json.load(file)
with open('../Categorie/shop_nonessenziale.json') as file:
    cat_shop_nonessenziale = json.load(file)
#json AMENITY
with open('../Categorie/ristoro.json') as file:
    cat_ristoro = json.load(file)
with open('../Categorie/servizi.json') as file:
    cat_servizi = json.load(file)
with open('../Categorie/tempolibero.json') as file:
    cat_tempolibero = json.load(file)
#json LEISURE
with open('../Categorie/sports.json') as file:
    cat_sports = json.load(file)
with open('../Categorie/turismo.json') as file:
    cat_turismo = json.load(file)
#json BUILDING
with open('../Categorie/lavoro.json') as file:
    cat_lavoro = json.load(file)
with open('../Categorie/scuola.json') as file:
    cat_scuola = json.load(file)

____________

In [7]:
#POI_shop_essenziale
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_shop_essenziale=[]

#Lista categorie
cat=[]
for se in cat_shop_essenziale:
    cat.append(se['shop'])
    
#Definisco Thread
def setup_poi_shop_essenziale(f):
    global mutex
    global id_num
    global POI_shop_essenziale
    global cat
    POI={}
    
    if f['properties']['shop'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['shop']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('shop')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto

        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Estraggo livelli edificio 

        try:    
            POI['lvl_edificio']=f['properties']['building:levels']
        except:
            POI['lvl_edificio']='1'

        #Stima numero dipendenti
        info=list(filter(lambda info: info['shop'] == f['properties']['shop'], cat_shop_essenziale))
    
        POI['dipendenti']=int(info[0]['dipendenti'])*int(POI['lvl_edificio'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria
        
        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_shop_essenziale.append(POI)

            #Aggiungo record
            mutex.release()

#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for s in shop:
    t=Thread(target=setup_poi_shop_essenziale, args=(s,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()
    
print('POI_shop_essenziale creato - Numero POI: '+str(len(POI_shop_essenziale)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:44:13
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_shop_essenziale creato - Numero POI: 2835
Fine = 2021-03-24 15:46:09


In [8]:
with open('./POI_shop_essenziale.json', 'w') as fp:
    json.dump(POI_shop_essenziale,fp)

______________

In [9]:
#POI_shop_nonessenziale
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_shop_nonessenziale=[]

#Lista categorie
cat=[]
for sne in cat_shop_nonessenziale:
    cat.append(sne['shop'])
    
#Definisco Thread
def setup_poi_shop_nonessenziale(f):
    global mutex
    global id_num
    global POI_shop_nonessenziale
    global cat
    POI={}
    
    if f['properties']['shop'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['shop']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('shop')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto

        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Estraggo livelli edificio 

        try:    
            POI['lvl_edificio']=f['properties']['building:levels']
        except:
            POI['lvl_edificio']='1'

        #Stima numero dipendenti
        info=list(filter(lambda info: info['shop'] == f['properties']['shop'], cat_shop_nonessenziale))
    
        POI['dipendenti']=int(info[0]['dipendenti'])*int(POI['lvl_edificio'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria
        
        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_shop_nonessenziale.append(POI)

            #Aggiungo record
            mutex.release()

#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for s in shop:
    t=Thread(target=setup_poi_shop_nonessenziale, args=(s,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()
    
print('POI_shop_nonessenziale creato - Numero POI: '+str(len(POI_shop_nonessenziale)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:46:09
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_shop_nonessenziale creato - Numero POI: 2307
Fine = 2021-03-24 15:47:16


In [10]:
with open('./POI_shop_nonessenziale.json', 'w') as fp:
    json.dump(POI_shop_nonessenziale,fp)

___________

In [12]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_ristoro=[]

#Lista categorie
cat=[]
for r in cat_ristoro:
    cat.append(r['amenity'])

#Definisco Thread
def setup_record_ristoro(f):
    POI={}
    
    global POI_ristoro
    global id_num
    global mutex
    global cat
    
    if f['properties']['amenity'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['amenity']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('amenity')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['amenity'] == f['properties']['amenity'], cat_ristoro))

        POI['dipendenti']=int(info[0]['dipendenti'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_ristoro.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for a in amenity:
    t=Thread(target=setup_record_ristoro, args=(a,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_ristoro aggiornato - Numero POI: '+str(len(POI_ristoro)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:47:16
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_ristoro aggiornato - Numero POI: 4479
Fine = 2021-03-24 15:50:12


In [13]:
with open('./POI_ristoro.json', 'w') as fp:
    json.dump(POI_ristoro,fp)

____________

In [14]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_servizi=[]

#Lista categorie
cat=[]
for r in cat_servizi:
    cat.append(r['amenity'])

#Definisco Thread
def setup_record_servizi(f):
    POI={}
    
    global POI_servizi
    global id_num
    global mutex
    global cat
    
    if f['properties']['amenity'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['amenity']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('amenity')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['amenity'] == f['properties']['amenity'], cat_servizi))

        POI['dipendenti']=int(info[0]['dipendenti'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_servizi.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for a in amenity:
    t=Thread(target=setup_record_servizi, args=(a,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_servizi aggiornato - Numero POI: '+str(len(POI_servizi)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:50:13
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_servizi aggiornato - Numero POI: 1547
Fine = 2021-03-24 15:51:20


In [15]:
with open('./POI_servizi.json', 'w') as fp:
    json.dump(POI_servizi,fp)

___________

In [17]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_tempolibero=[]

#Lista categorie
cat=[]
for r in cat_tempolibero:
    cat.append(r['amenity'])

#Definisco Thread
def setup_record_tempolibero(f):
    POI={}
    
    global POI_tempolibero
    global id_num
    global mutex
    global cat
    
    if f['properties']['amenity'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['amenity']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('amenity')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['amenity'] == f['properties']['amenity'], cat_tempolibero))

        POI['dipendenti']=int(info[0]['dipendenti'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_tempolibero.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for a in amenity:
    t=Thread(target=setup_record_tempolibero, args=(a,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_tempolibero aggiornato - Numero POI: '+str(len(POI_tempolibero)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:51:20
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_tempolibero aggiornato - Numero POI: 82
Fine = 2021-03-24 15:51:24


In [18]:
with open('./POI_tempolibero.json', 'w') as fp:
    json.dump(POI_tempolibero,fp)

____________

In [20]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_turismo=[]

#Lista categorie
cat=[]
for r in cat_turismo:
    cat.append(r['leisure'])

#Definisco Thread
def setup_record_turismo(f):
    POI={}
    
    global POI_turismo
    global id_num
    global mutex
    global cat
    
    if f['properties']['leisure'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['leisure']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('leisure')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['leisure'] == f['properties']['leisure'], cat_turismo))

        POI['dipendenti']=int(info[0]['dipendenti'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_turismo.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for l in leisure:
    t=Thread(target=setup_record_turismo, args=(l,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_turismo aggiornato - Numero POI: '+str(len(POI_turismo)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:51:24
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_turismo aggiornato - Numero POI: 1972
Fine = 2021-03-24 15:53:04


In [21]:
with open('./POI_turismo.json', 'w') as fp:
    json.dump(POI_turismo,fp)

____________

In [22]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_sports=[]

#Lista categorie
cat=[]
for r in cat_sports:
    cat.append(r['leisure'])

#Definisco Thread
def setup_record_sports(f):
    POI={}
    
    global POI_sports
    global id_num
    global mutex
    global cat
    
    if f['properties']['leisure'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['leisure']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('leisure')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['leisure'] == f['properties']['leisure'], cat_sports))

        POI['dipendenti']=int(info[0]['dipendenti'])

        #Stima capienza_oraria
        capienza_oraria=[]
        for i in range(0,13):
            capienza_oraria.append(int(info[0]['capienza_oraria']))
        POI['capienza_oraria']=capienza_oraria

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_sports.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for l in leisure:
    t=Thread(target=setup_record_sports, args=(l,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_sports aggiornato - Numero POI: '+str(len(POI_sports)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 15:53:04
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_sports aggiornato - Numero POI: 1390
Fine = 2021-03-24 15:54:22


In [23]:
with open('./POI_sports.json', 'w') as fp:
    json.dump(POI_sports,fp)

____________

In [19]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_lavoro=[]

#Lista categorie
cat=[]
for l in cat_lavoro:
    cat.append(l['building'])

#Definisco Thread
def setup_record_lavoro(f):
    POI={}
    
    global POI_lavoro
    global id_num
    global mutex
    global cat
    
    if f['properties']['building'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['building']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('building')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['building'] == f['properties']['building'], cat_lavoro))

        POI['dipendenti']=int(info[0]['dipendenti'])
        
        POI['essenziale']=True

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_lavoro.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for b in building:
    t=Thread(target=setup_record_lavoro, args=(b,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_lavoro aggiornato - Numero POI: '+str(len(POI_lavoro)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-25 10:06:50
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_lavoro aggiornato - Numero POI: 3839
Fine = 2021-03-25 10:10:05


In [20]:
with open('./POI_lavoro.json', 'w') as fp:
    json.dump(POI_lavoro,fp)

____________

In [52]:
#Funzione assegna capienza scuola
def capienza_scuola(tipo):
    switcher={
            'college':300,
            'school':200,
            'university':350,
            'kindergarten':150
         }
    return switcher.get(tipo,"errore")

In [53]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_scuola=[]

#Lista categorie
cat=[]
for s in cat_scuola:
    cat.append(s['building'])

#Definisco Thread
def setup_record_scuola(f):
    POI={}
    
    global POI_scuola
    global id_num
    global mutex
    global cat
    
    if f['properties']['building'] in cat:
        #Estraggo il nome
        try:
            POI['nome']=f['properties']['name']
        except:
            try:
                POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
            except:
                POI['nome']=''

        #Estraggo tipologia
        try:    
            POI['tipo_edificio']=f['properties']['building']
        except:
            try:
                POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('building')
            except:
                POI['tipo_edificio']=''

        #Ripopolo campo 'Nome' se vuoto
        if POI['nome']=='':
            POI['nome']=POI['tipo_edificio'].capitalize()

        #Stima numero dipendenti
        info=list(filter(lambda info: info['building'] == f['properties']['building'], cat_scuola))

        POI['dipendenti']=int(info[0]['dipendenti'])
        
        #Assegno capienza
        
        POI['capienza']=capienza_scuola(f['properties']['building'])

        #Estraggo coordinate
        if f['geometry']['type']=='Point':
            coord_X=f['geometry']['coordinates'][0]
            coord_Y=f['geometry']['coordinates'][1]       
        else:
            if f['geometry']['type']=='LineString':
                coord_X=f['geometry']['coordinates'][0][0]
                coord_Y=f['geometry']['coordinates'][0][1]
            else:
                coord_X=f['geometry']['coordinates'][0][0][0]
                coord_Y=f['geometry']['coordinates'][0][0][1]

        #Correggo errore dati - coordinata come lista
        if isinstance(coord_X, list):
            POI['coord_X']=coord_X[0]
            POI['coord_Y']=coord_X[1]
        else:
            POI['coord_X']=coord_X
            POI['coord_Y']=coord_Y

        #Assegno zona censo
        POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

        #Salvo il dict
        if POI['sezione_censo'] is not None:

            mutex.acquire()

            #Assegno ID
            POI['id']=id_num
            id_num=id_num+1

            POI_scuola.append(POI)

            #Aggiungo record
            mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for b in building:
    t=Thread(target=setup_record_scuola, args=(b,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_scuola aggiornato - Numero POI: '+str(len(POI_scuola)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-23 11:28:59
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_scuola aggiornato - Numero POI: 562
Fine = 2021-03-23 11:29:30


In [54]:
with open('./POI_scuola.json', 'w') as fp:
    json.dump(POI_scuola,fp)

____________

In [4]:
#Re-importo POI
with open('./POI_lavoro.json') as json_file:
    POI_lavoro = json.load(json_file)
with open('./POI_sports.json') as json_file:
    POI_sports = json.load(json_file)
with open('./POI_turismo.json') as json_file:
    POI_turismo = json.load(json_file)
with open('./POI_tempolibero.json') as json_file:
    POI_tempolibero = json.load(json_file)
with open('./POI_servizi.json') as json_file:
    POI_servizi = json.load(json_file)
with open('./POI_ristoro.json') as json_file:
    POI_ristoro = json.load(json_file)
with open('./POI_shop_essenziale.json') as json_file:
    POI_shop_essenziale = json.load(json_file)
with open('./POI_shop_nonessenziale.json') as json_file:
    POI_shop_nonessenziale = json.load(json_file)
with open('./POI_scuola.json') as json_file:
    POI_scuola = json.load(json_file)

In [5]:
#Creo json_posti_lavoro
with open('./json_posti_lavoro.json', 'w') as fp:
    POI_posti_lavoro=[]
    for r in POI_lavoro:
        r['capienza']=r['dipendenti']
        POI_posti_lavoro.append(r)
    for r in POI_ristoro:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=False
        POI_posti_lavoro.append(r)
    for r in POI_turismo:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=False
        POI_posti_lavoro.append(r)
    for r in POI_shop_essenziale:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=True
        POI_posti_lavoro.append(r)
    for r in POI_shop_nonessenziale:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=False
        POI_posti_lavoro.append(r)
    for r in POI_servizi:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=True
        POI_posti_lavoro.append(r)
    for r in POI_tempolibero:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=False
        POI_posti_lavoro.append(r)
    for r in POI_sports:
        r['capienza']=r['capienza_oraria'][0]
        r['essenziale']=False
        POI_posti_lavoro.append(r)
    for r in POI_scuola:
        r['capienza']=r['capienza']
        r['essenziale']=False
        POI_posti_lavoro.append(r)
        
    json.dump(POI_posti_lavoro, fp)

In [8]:
#Correzione Coordinate POI - Errori su type delle coordinate
import collections

with open('./json_posti_lavoro.json') as json_file:
    posti_lavoro = json.load(json_file)
    
coord_X=[]
for p in posti_lavoro:
    coord_X.append(type(p['coord_X']))

print(collections.Counter(coord_X))

coord_Y=[]
for p in posti_lavoro:
    coord_Y.append(type(p['coord_Y']))

print(collections.Counter(coord_Y))

Counter({<class 'float'>: 19013})
Counter({<class 'float'>: 19013})
